<a href="https://colab.research.google.com/github/kdemertzis/LNexamples/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam


In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 45
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [ ]:
import tensorflow as tf

In [ ]:
num_classes = 10

In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
print(X_test.shape)
X_train.shape

(10000, 32, 32, 3)


(50000, 32, 32, 3)

In [ ]:
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
  global compression
  temp = input
  for _ in range(l):
    BatchNorm = layers.BatchNormalization()(temp)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
    if (dropout_rate>0):
      Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
      concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
    temp = concat
        
  return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
  global compression
  BatchNorm = layers.BatchNormalization()(input)
  relu = layers.Activation('relu')(BatchNorm)
  Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
  if dropout_rate>0:
    Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
  avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
  return avg

#output layer
def output_layer(input):
  global compression
  BatchNorm = layers.BatchNormalization()(input)
  relu = layers.Activation('relu')(BatchNorm)
  AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
  flat = layers.Flatten()(AvgPooling)
  output = layers.Dense(num_classes, activation='softmax')(flat)
  return output

In [ ]:
X_train[0].shape

(32, 32, 3)

In [ ]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 12)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
DynamicLR =ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=2, verbose=1, mode='max',min_delta=0, cooldown=0, min_lr=0.0001)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, 
                    validation_data=(X_test, y_test),callbacks=[DynamicLR])

Epoch 1/45
391/391 [==============================] - 37s 65ms/step - loss: 1.9166 - accuracy: 0.2739 - val_loss: 2.3090 - val_accuracy: 0.2654
Epoch 2/45
391/391 [==============================] - 24s 61ms/step - loss: 1.4206 - accuracy: 0.4746 - val_loss: 1.8183 - val_accuracy: 0.4207
Epoch 3/45
391/391 [==============================] - 24s 61ms/step - loss: 1.2186 - accuracy: 0.5545 - val_loss: 1.3991 - val_accuracy: 0.5323
Epoch 4/45
327/391 [========================>.....] - ETA: 3s - loss: 1.1046 - accuracy: 0.6035

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

DATA AUGMENTATION

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [ ]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model1 = Model(inputs=[input], outputs=[output])
model1.summary()

In [ ]:
DynamicLR =ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=2, verbose=1, mode='max',min_delta=0, cooldown=0, min_lr=0.0001)

In [ ]:
model1.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

In [ ]:
history2=model1.fit_generator(datagen.flow(X_train,y_train,batch_size=128),validation_data = (X_test,y_test), steps_per_epoch = 391,epochs=45,callbacks=[DynamicLR])

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()